<a href="https://colab.research.google.com/github/ceofast/vLLM-vs-Transformers/blob/main/vllm-vs-transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Tue Dec 12 20:08:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.1 --progress-bar off
!pip install -qqq transformers==4.35.2 --progress-bar off
!pip install -qqq accelerate==0.24.1 --progress-bar off
!pip install -qqq vllm==0.2.2 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
torchaudio 2.1.0+cu118 requires torch==2.1.0

In [4]:
from pathlib import Path
from huggingface_hub import notebook_login
from tqdm import tqdm

notebook_login()

In [6]:
import torch
from transformers import PreTrainedTokenizer, pipeline


text_pipeline = pipeline("text-generation",
                         model="meta-llama/Llama-2-7b-chat-hf",
                         torch_dtype = torch.float16,
                         device_map="auto")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
texts = ["What are the pros/cons of ChatGPT vs Open Source LLMs?",
         "Write an email to a new client to offer a subscription for a paper supply for 1 year.",
         "I have $10.000 USD for investment. How one should invest it during times of high inflation and high mortgate rate",
         "Write a function in python that calculates the square of a sum of two numbers."]

In [9]:
def create_prompt(text: str, tokenizer: PreTrainedTokenizer) -> str:
  messages = [
      {
          "role": "system",
          "content": "You are a friendly chatbot who always responds as superhuman intelligence AI.",
      },
      {"role": "user", "content": text},
  ]
  return tokenizer.apply_chat_template(
      messages, tokenize = False, add_generation_prompt = True
  )

In [10]:
prompts = [create_prompt(text, text_pipeline.tokenizer) for text in texts]

In [11]:
for prompt in prompts:
  print(prompt)
  print()
  print()

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]


<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

Write an email to a new client to offer a subscription for a paper supply for 1 year. [/INST]


<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

I have $10.000 USD for investment. How one should invest it during times of high inflation and high mortgate rate [/INST]


<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

Write a function in python that calculates the square of a sum of two numbers. [/INST]




In [12]:
folder_path = Path("prompts")
folder_path.mkdir(parents = True, exist_ok = True)

for i, prompt in enumerate(prompts, start = 1):
  file_path = folder_path / f"prompt_{i}.txt"
  file_path.write_text(prompt)

In [14]:
%%time
outputs = text_pipeline(prompts[0], max_new_tokens = 256, do_sample = True, temperature = 0.1)

CPU times: user 11.7 s, sys: 635 ms, total: 12.3 s
Wall time: 12.3 s


In [15]:
print(outputs[0]["generated_text"])

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]  Greetings! I'm here to help you compare ChatGPT and Open Source LLMs. Both are powerful language models, but they have different strengths and weaknesses. Here are some pros and cons of each:

ChatGPT:
Pros:

1. High-quality responses: ChatGPT is trained on a massive dataset and has been fine-tuned for a wide range of tasks, resulting in high-quality responses that are often indistinguishable from those generated by a human.
2. Easy to use: ChatGPT has a user-friendly interface that makes it simple to integrate into various applications, such as chatbots, virtual assistants, and language translation tools.
3. Customizable: ChatGPT can be fine-tuned for specific tasks and domains, allowing developers to tailor the model to their specific needs.
4. Scalable: ChatGPT can handle a large volume of requests and can be easily

In [16]:
### vLLM Inference

In [17]:
from pathlib import Path
import torch
from vllm import LLM, SamplingParams

In [18]:
folder_path = Path("prompts")
prompts = []
for file_path in sorted(list(folder_path.glob("*.txt"))):
  content = file_path.read_text()
  prompts.append(content)

In [19]:
print(prompts[0])

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]


In [20]:
llm = LLM(model = "meta-llama/Llama-2-7b-chat-hf", dtype = torch.float16)

INFO 12-12 20:30:28 llm_engine.py:72] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 12-12 20:30:28 tokenizer.py:31] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.
INFO 12-12 20:30:37 llm_engine.py:207] # GPU blocks: 1270, # CPU blocks: 512


In [21]:
sampling_params = SamplingParams(temperature = 0.1, max_tokens = 256)

In [22]:
%%time
outputs = llm.generate(prompts[0], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.47s/it]

CPU times: user 3.49 s, sys: 0 ns, total: 3.49 s
Wall time: 3.48 s


In [23]:
for output in outputs:
  generated_text = output.outputs[0].text
  print(generated_text)

  Greetings! I'm here to help you compare the pros and cons of ChatGPT and Open Source LLMs. Both are powerful language models, but they have different strengths and weaknesses. Here's a comprehensive comparison:

ChatGPT:
Pros:

1. High-quality responses: ChatGPT is trained on a massive dataset and has been fine-tuned for a wide range of tasks, resulting in high-quality responses that are often indistinguishable from human language.
2. Easy to use: ChatGPT has a simple and intuitive API that makes it easy to integrate into various applications, such as chatbots, voice assistants, and language translation tools.
3. Fast response times: ChatGPT can generate responses quickly, even for long conversations, thanks to its parallel processing capabilities.
4. Customizable: ChatGPT can be fine-tuned for specific tasks and domains, allowing developers to tailor the model to their needs.

Cons:

1. Limited domain knowledge: While ChatGPT has been trained on a vast dataset, its knowledge is limi

In [24]:
%%time
hf_batch_outputs = text_pipeline(prompts, max_new_tokens = 256, do_sample = True, temperature = 0.1)

CPU times: user 38.8 s, sys: 15.7 ms, total: 38.8 s
Wall time: 38.7 s


In [25]:
for output in hf_batch_outputs:
  print(output[0]["generated_text"])
  print()

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI.
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]  Ah, an excellent question! *adjusts glasses* ChatGPT and Open Source LLMs are both powerful language models, but they have some key differences in their architecture, training, and usage. Here are some pros and cons of each:

ChatGPT:
Pros:

1. **Customization**: ChatGPT is a highly customizable model that can be fine-tuned for specific tasks, such as chatbots, language translation, or text generation.
2. **Faster Training**: ChatGPT is trained on a larger dataset than Open Source LLMs, which allows it to learn faster and adapt to new tasks more quickly.
3. **Better Handling of Out-of-Vocabulary Words**: ChatGPT is trained on a wider range of texts, including web pages, articles, and books, which helps it handle out-of-vocabulary words better than Open Source LLMs.
4. **Improved Contextual Understanding**: ChatGPT's ar

In [26]:
%%time
vllm_batch_outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]

CPU times: user 3.56 s, sys: 0 ns, total: 3.56 s
Wall time: 3.55 s


In [27]:
for output in vllm_batch_outputs:
  generated_text = output.outputs[0].text
  print(generated_text)
  print()

  Greetings! I'm here to help you compare the pros and cons of ChatGPT and Open Source Large Language Models (LLMs). Both are powerful language models that have revolutionized the field of natural language processing, but they have some key differences.

**ChatGPT:**

Pros:

1. **Ease of use:** ChatGPT is a user-friendly interface that allows you to interact with the model using natural language. You don't need to have any technical knowledge to use it.
2. **Faster response times:** ChatGPT is designed to respond quickly and efficiently, allowing you to get answers to your questions in real-time.
3. **Customizable:** ChatGPT can be customized to fit your specific needs and use cases, making it a versatile tool for a wide range of applications.
4. **High accuracy:** ChatGPT has been trained on a large dataset and has demonstrated high accuracy in understanding and responding to natural language inputs.

Cons:

1. **Limited flexibility:** While ChatGPT can be customized, it is still

  S